### Activity: Text Extractor with Map-Reduce
- Francisco Javier De la Torre Silva
- Edgar Alonso Zaragoza Delgadillo

11/02/2026

In [1]:
!pip3 install pytesseract pillow opencv-python

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from PIL import Image
import pytesseract

In [3]:
spark = SparkSession.builder \
        .appName("1st Spark Session") \
        .master("local[*]") \
        .config("spark.ui.port","4040") \
        .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/12 01:44:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/12 01:44:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
def extract_text(img_path):
    img = Image.open(img_path)
    return pytesseract.image_to_string(img)

In [5]:
extract_text("../assets/close_sign.png")

' \n\x0c'

In [6]:
base_path = "../assets/"
imgs_paths = ["close_sign.png", "night_sign.png", "stop_sign.png", "open_sign.png"]
imgs_paths_wa = [base_path+x for x in imgs_paths]

[extract_text(x) for x in imgs_paths_wa]

[' \n\x0c', 'NIGHT\n\x0c', ' \n\x0c', ' \n\x0c']

In [7]:
parallelized_images = sc.parallelize(imgs_paths_wa)
text_rdd = parallelized_images.map(extract_text)
text_lst = text_rdd.collect()
text_lst

[' \n\x0c', 'NIGHT\n\x0c', ' \n\x0c', ' \n\x0c']